In [115]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MultiLabelBinarizer


In [116]:

# Charger les données
data = pd.read_csv('datasets/games_data.csv', sep=',')

In [117]:
useless_columns = [
    'detailed_description',  # Trop textuelle, inutile sauf analyse de texte
    'short_description',     # Résumé court, pas toujours nécessaire
    'reviews',               # Si vous ne voulez pas analyser les avis détaillés
    'header_image',          # Contient des URLs d'images, souvent non utiles
    'website',               # Site officiel, souvent inutile pour l'analyse
    'support_url',           # URL de support, inutile si l'analyse ne concerne pas l'assistance
    'support_email',         # Email de support, inutile sauf analyse dédiée
    'score_rank',            # Redondant avec `user_score` ou `metacritic_score`
    'achievements',          # Non essentiel sauf analyse des réalisations
    'recommendations',       # Non essentiel sauf analyse des recommandations
    'notes',                 # Informations supplémentaires qui ne sont pas cruciales
    'screenshots',           # URLs d'images, inutiles pour des analyses numériques
    'movies',                # URLs de vidéos, inutiles pour des analyses numériques
    'full_audio_languages',
    'appID',  
    'user_score', 
    'metacritic_url'
]

dataset_clean = data.drop(columns=useless_columns)
print(dataset_clean.columns)
print(dataset_clean.shape)

Index(['name', 'release_date', 'estimated_owners', 'peak_ccu', 'required_age',
       'price', 'dlc_count', 'languages', 'support_windows', 'support_mac',
       'support_linux', 'metacritic_score', 'positive_reviews',
       'negative_reviews', 'average_playtime_forever',
       'average_playtime_2weeks', 'median_playtime_forever',
       'median_playtime_2weeks', 'developers', 'publishers', 'categories',
       'genres', 'tags'],
      dtype='object')
(97410, 23)


In [118]:
dataset_clean.head(10)

,name,release_date,estimated_owners,peak_ccu,required_age,price,dlc_count,languages,support_windows,support_mac,...,negative_reviews,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,developers,publishers,categories,genres,tags
0,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,English,True,False,...,11,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player, Multi-player, Steam Achievement...","Casual, Indie, Sports","Indie, Casual, Sports, Bowling"
1,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,"English, French, Italian, German, Spanish - Sp...",True,True,...,5,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player, Steam Achievements, Full contro...","Action, Indie","Indie, Action, Pixel Graphics, 2D, Retro, Arca..."
2,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,"English, Portuguese - Brazil",True,False,...,0,0,0,0,0,Campião Games,Campião Games,Single-player,"Action, Adventure, Indie, Strategy",NaN
3,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,"English, French, Italian, German, Spanish - Sp...",True,True,...,0,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player, Full controller support","Adventure, Casual, Indie","2D Platformer, Atmospheric, Surreal, Mystery, ..."
4,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,"English, Spanish - Spain",True,True,...,8,0,0,0,0,Unusual Games,Unusual Games,"Single-player, Steam Achievements","Adventure, Indie","Indie, Adventure, Nudity, Violent, Sexual Cont..."
5,Wartune Reborn,"Feb 26, 2021",50000 - 100000,68,0,0.00,0,English,True,False,...,49,0,0,0,0,7Road,7Road,"Single-player, Multi-player, MMO, PvP, Online ...","Adventure, Casual, Free to Play, Massively Mul...","Turn-Based Combat, Massively Multiplayer, Mult..."
6,TD Worlds,"Jan 9, 2022",0 - 20000,3,0,10.99,1,"English, Russian, Danish",True,False,...,7,0,0,0,0,MAKSIM VOLKAU,MAKSIM VOLKAU,"Single-player, Steam Achievements, Steam Cloud","Indie, Strategy","Tower Defense, Rogue-lite, RTS, Replay Value, ..."
7,Legend of Rome - The Wrath of Mars,"May 5, 2022",0 - 20000,2,0,9.99,0,"English, German",True,False,...,0,0,0,0,0,magnussoft,magnussoft,"Single-player, Steam Cloud",Casual,NaN
8,MazM: Jekyll and Hyde,"Apr 2, 2020",0 - 20000,1,0,14.99,0,"English, French, Italian, German, Spanish - Sp...",True,False,...,6,0,0,0,0,Growing Seeds,"CFK Co., Ltd.","Single-player, Steam Achievements, Full contro...","Adventure, RPG, Simulation, Strategy","Adventure, Simulation, RPG, Strategy, Singlepl..."
9,Deadlings: Rotten Edition,"Nov 11, 2014",50000 - 100000,0,0,3.99,0,"English, Polish, French, Italian, German, Span...",True,True,...,45,703,0,782,0,ONE MORE LEVEL,ONE MORE LEVEL,"Single-player, Steam Achievements, Steam Tradi...","Action, Adventure, Indie","Action, Indie, Adventure, Puzzle-Platformer, A..."


In [119]:
# Convertir 'release_date' en datetime
dataset_clean['release_date'] = pd.to_datetime(dataset_clean['release_date'], errors='coerce')

# Créer une nouvelle colonne pour l'âge du jeu en années
dataset_clean['age_of_game'] = (pd.Timestamp.now() - dataset_clean['release_date']).dt.days / 365


In [120]:
# Fonction pour calculer la médiane des plages
def extract_median(range_str):
    if pd.isna(range_str):
        return None
    try:
        min_val, max_val = map(int, range_str.split(' - '))
        return (min_val + max_val) / 2
    except:
        return None

dataset_clean['estimated_owners'] = dataset_clean['estimated_owners'].apply(extract_median)


In [121]:
# Compter le nombre de langues supportées
dataset_clean['nb_languages'] = dataset_clean['languages'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0)


In [122]:
dataset_clean['nb_languages'] 

0         1
1        10
2         2
3        11
4         2
         ..
97405     2
97406    11
97407     1
97408     1
97409     1
Name: nb_languages, Length: 97410, dtype: int64

In [123]:
dataset_clean.shape

(97410, 25)

In [124]:
# Remplir les colonnes numériques par la médiane
num_cols = ['estimated_owners', 'metacritic_score', 'positive_reviews', 'negative_reviews', 'age_of_game']
for col in num_cols:
    dataset_clean[col].fillna(dataset_clean[col].median(), inplace=True)

# Remplir les colonnes textuelles avec 'Unknown'
text_cols = ['name', 'developers', 'publishers', 'categories', 'genres', 'tags']
for col in text_cols:
    dataset_clean[col].fillna('Unknown', inplace=True)

# Remplir les colonnes booléennes avec False
bool_cols = ['support_windows', 'support_mac', 'support_linux']
for col in bool_cols:
    dataset_clean[col].fillna(False, inplace=True)

In [125]:
# Compter le nombre de NaN par colonne
dataset_clean = dataset_clean.dropna()


nan_count = dataset_clean.isna().sum()

# Afficher le résultat
print(nan_count)
print(dataset_clean.shape)

name                        0
release_date                0
estimated_owners            0
peak_ccu                    0
required_age                0
price                       0
dlc_count                   0
languages                   0
support_windows             0
support_mac                 0
support_linux               0
metacritic_score            0
positive_reviews            0
negative_reviews            0
average_playtime_forever    0
average_playtime_2weeks     0
median_playtime_forever     0
median_playtime_2weeks      0
developers                  0
publishers                  0
categories                  0
genres                      0
tags                        0
age_of_game                 0
nb_languages                0
dtype: int64
(92449, 25)


In [126]:
features = [
    'estimated_owners', 'peak_ccu', 'required_age', 'dlc_count', 'support_windows',
    'support_mac', 'support_linux', 'metacritic_score', 'positive_reviews',
    'negative_reviews', 'average_playtime_forever', 'age_of_game', 'nb_languages'
]
target = 'price'

In [127]:
# Encodage OneHot pour les colonnes booléennes
encoder = OneHotEncoder(drop='if_binary', sparse=False)

# Ajouter un préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features),  # Normaliser les colonnes numériques
    ],
    remainder='passthrough'
)


In [128]:
dataset_clean.dtypes

name                                object
release_date                datetime64[ns]
estimated_owners                   float64
peak_ccu                             int64
required_age                         int64
price                              float64
dlc_count                            int64
languages                           object
support_windows                       bool
support_mac                           bool
support_linux                         bool
metacritic_score                     int64
positive_reviews                     int64
negative_reviews                     int64
average_playtime_forever             int64
average_playtime_2weeks              int64
median_playtime_forever              int64
median_playtime_2weeks               int64
developers                          object
publishers                          object
categories                          object
genres                              object
tags                                object
age_of_game

In [129]:
# Identifier les colonnes non numériques
non_numeric_columns = dataset_clean.select_dtypes(include=['object']).columns
print("Colonnes non numériques : ", non_numeric_columns)

Colonnes non numériques :  Index(['name', 'languages', 'developers', 'publishers', 'categories', 'genres',
       'tags'],
      dtype='object')


In [130]:
# Assurer que ces colonnes sont de type booléen
bool_columns = ['support_windows', 'support_mac', 'support_linux']
for col in bool_columns:
    dataset_clean[col] = dataset_clean[col].astype(bool)

In [131]:
# Pour 'languages' : créer une liste de langues pour chaque jeu
dataset_clean['languages_list'] = dataset_clean['languages'].apply(lambda x: x.split(', ') if pd.notna(x) else [])

# Encoder les langues avec MultiLabelBinarizer
mlb_languages = MultiLabelBinarizer()
languages_encoded = mlb_languages.fit_transform(dataset_clean['languages_list'])
languages_encoded_df = pd.DataFrame(languages_encoded, columns=mlb_languages.classes_)

# Ajouter ces nouvelles colonnes au dataset
dataset_clean = pd.concat([dataset_clean, languages_encoded_df], axis=1)


In [132]:
# Tags
dataset_clean['tags_list'] = dataset_clean['tags'].apply(lambda x: x.split(', ') if pd.notna(x) else [])
mlb_tags = MultiLabelBinarizer()
tags_encoded = mlb_tags.fit_transform(dataset_clean['tags_list'])
tags_encoded_df = pd.DataFrame(tags_encoded, columns=mlb_tags.classes_)
dataset_clean = pd.concat([dataset_clean, tags_encoded_df], axis=1)

# Genres
dataset_clean['genres_list'] = dataset_clean['genres'].apply(lambda x: x.split(', ') if pd.notna(x) else [])
mlb_genres = MultiLabelBinarizer()
genres_encoded = mlb_genres.fit_transform(dataset_clean['genres_list'])
genres_encoded_df = pd.DataFrame(genres_encoded, columns=mlb_genres.classes_)
dataset_clean = pd.concat([dataset_clean, genres_encoded_df], axis=1)

# Categories
dataset_clean['categories_list'] = dataset_clean['categories'].apply(lambda x: x.split(', ') if pd.notna(x) else [])
mlb_categories = MultiLabelBinarizer()
categories_encoded = mlb_categories.fit_transform(dataset_clean['categories_list'])
categories_encoded_df = pd.DataFrame(categories_encoded, columns=mlb_categories.classes_)
dataset_clean = pd.concat([dataset_clean, categories_encoded_df], axis=1)

In [133]:
# Supprimer les colonnes non numériques si elles ne sont pas nécessaires
dataset_clean = dataset_clean.drop(columns=['name', 'release_date', 'languages', 'developers', 'publishers', 'categories', 'genres', 'tags', 'languages_list', 'tags_list', 'genres_list', 'categories_list'])

In [134]:
# Remplir les NaN par False (ou 0) et convertir en int
dataset_clean['support_windows'] = dataset_clean['support_windows'].fillna(False).astype(int)
dataset_clean['support_mac'] = dataset_clean['support_mac'].fillna(False).astype(int)
dataset_clean['support_linux'] = dataset_clean['support_linux'].fillna(False).astype(int)


In [135]:
# Sélectionner uniquement les colonnes non numériques
non_numeric_columns = dataset_clean.select_dtypes(exclude=['number']).columns
print(non_numeric_columns)


Index([], dtype='object')


In [138]:
# Afficher le nombre de valeurs NaN par colonne
nan_count = dataset_clean.isna().sum()

# Afficher les colonnes contenant des NaN
print(nan_count[nan_count > 0])

estimated_owners            4960
peak_ccu                    4960
required_age                4960
price                       4960
dlc_count                   4960
                            ... 
Unknown                        5
VR Only                        5
VR Support                     5
VR Supported                   5
Valve Anti-Cheat enabled       5
Length: 680, dtype: int64


In [139]:
# Supprimer les lignes contenant des valeurs NaN
dataset_clean = dataset_clean.dropna()

# Vérifier que les NaN ont bien été supprimés
print(dataset_clean.isna().sum())

estimated_owners            0
peak_ccu                    0
required_age                0
price                       0
dlc_count                   0
                           ..
Unknown                     0
VR Only                     0
VR Support                  0
VR Supported                0
Valve Anti-Cheat enabled    0
Length: 683, dtype: int64


In [ ]:
dataset_clean.shape

In [137]:
dataset_clean.to_csv('datasets/cleaned_games_data.csv', index=False)

print("Dataset nettoyé sauvegardé avec succès !")

Dataset nettoyé sauvegardé avec succès !
